# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound
from tqdm import tqdm
tqdm.pandas()

In [2]:
def get_tick_time(data):
    data['df']['sno'] = data['df'].index
    data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
    data['df']['DateTime_frmt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data["df"]['DateTime']]
    data['df'] = data['df'][['sno', 'DateTime_frmt', 'tick']]
    return(data)

In [3]:
def get_ohlc(data):
    
    data['df_bars']['sno'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').sno.first().values.astype(int)
    #data['df_bars']['ts'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.min().index
    data['df_bars']['o'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.first().values
    data['df_bars']['h'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.max().values
    data['df_bars']['l'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.min().values
    data['df_bars']['c'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.last().values
    data['df_bars'] = data['df_bars'].reset_index(drop=True)    
    return(data)

In [4]:
def get_sup_res(data):
    for i in tqdm(range(2, len(data['df_bars']))):
        if 1 < i < len(data['df_bars']) - 2:
            if data['df_bars']['l'][i] < data['df_bars']['l'][i-1] < data['df_bars']['l'][i-2] and data['df_bars']['l'][i] < data['df_bars']['l'][i+1] < data['df_bars']['l'][i+2]:
                data['df_bars']['support'][i] = 'yes'

            if data['df_bars']['h'][i] > data['df_bars']['h'][i-1] > data['df_bars']['h'][i-2] and data['df_bars']['h'][i] > data['df_bars']['h'][i+1] > data['df_bars']['h'][i+2]:
                data['df_bars']['resistance'][i] = 'yes'
    return(data)

In [5]:
%%time
data['df'] = pd.read_csv('data/EURUSD_2021.csv', nrows=1000)

Wall time: 7.98 ms


In [6]:
data['candle_size'] = '1T'
data['df_bars'] = pd.DataFrame()
data['df_bars']['support'] = ''
data['df_bars']['resistance'] = ''

In [7]:
data = get_tick_time(data)
data = get_ohlc(data)
data = get_sup_res(data)

100%|████████████████████████████████████████| 59/59 [00:00<00:00, 8758.55it/s]


In [8]:
data['df'] = pd.merge(data['df'], data['df_bars'], on = 'sno', how = 'left')

In [9]:
data['df'].head()

,sno,DateTime_frmt,tick,support,resistance,o,h,l,c
0,0,2021-01-03 22:00:00,1.224035,NaN,NaN,1.224035,1.224035,1.223835,1.224005
1,1,2021-01-03 22:00:00,1.224025,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2021-01-03 22:00:00,1.224020,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2021-01-03 22:00:00,1.224015,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2021-01-03 22:00:00,1.224005,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
data['support_values'] = list(data['df']['l'][data['df']['support'] == 'yes'])
data['resistance_values'] = list(data['df']['h'][data['df']['resistance'] == 'yes'])

In [56]:
fig = px.line(data['df'], x="sno", y="tick", title='tick chart')
for val in data['support_values']:
    fig.add_hline(y = val, line_width=3, line_dash="dash", line_color="lightgreen")
for val in data['resistance_values']:
    fig.add_hline(y = val, line_width=3, line_dash="dot", line_color="darkred")
fig.show()